# Committor functions

In this notebook we will reproduce the figures and tables on skills of the gaussian committor and optimal projection patterns, applied to the ERA5 dataset

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib widget
matplotlib.rc('font', size=18)
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


import pandas as pd
import xarray as xr

from tqdm.notebook import tqdm

import sys
sys.path.append('../../../Climate-Learning/')

import general_purpose.utilities as ut
import general_purpose.cartopy_plots as cplt
import general_purpose.uplotlib as uplt
# import general_purpose.tables as tbl

# log to stdout
import logging
logging.getLogger().level = logging.INFO
logging.getLogger().handlers = [logging.StreamHandler(sys.stdout)]
ut.indentation_sep = '  '

HOME = '../../'

## Figure 9

In [ ]:
ds_comp = xr.open_dataarray('Skill-comp_T14_percent5.nc')
ds_comp

In [ ]:
ds_GA = xr.open_dataarray('Skill-GA_percent5_epsilonbest.nc').sel(T=14, tau=ds_comp['tau'])
ds_GA

In [ ]:
ds_CNN = xr.open_dataarray('Skill-CNN_T14_percent5.nc')
ds_CNN

In [ ]:
# average skill at T=14, tau=0, to put in the text
uplt.xr_avg(ds_GA.sel(tau=0), 'fold').data.item()

In [ ]:
np.stack([ds_GA.tau.data, ds_GA.reg_c.data.squeeze()]).T

In [ ]:
plt.close(9)
fig, ax = plt.subplots(num=9, figsize=(9,6))

offset = 0.15

uplt.plot(ds_GA['tau'] + offset, uplt.xr_avg(ds_GA, 'fold').data.squeeze(), label=r'GA $\epsilon_\mathrm{best}$')
uplt.plot(ds_comp['tau'] + 2*offset, uplt.xr_avg(ds_comp, 'fold').data.squeeze(), label='composite map')

uplt.errorband(ds_CNN['tau'], uplt.xr_avg(ds_CNN, 'fold').data.squeeze(), label='CNN')

plt.axhline(0, color='black', linestyle='dashed', label='climatology')
plt.xlabel(r'$\tau$ [days]')
plt.ylabel('Normalized log score')

ax.set_xlim(*ax.get_xlim())
ax.set_ylim(*ax.get_ylim())

plt.fill_between(ax.get_xlim(), ax.get_ylim()[0], 0, color='red', alpha=0.2)

plt.legend()

fig.tight_layout()


# fig.savefig(f'{HOME}/ERA_S_T14.pdf')

## Figure 10

In [ ]:
ds_PLASIM_Z = xr.open_dataarray('../../PLASIM/committor/Skill-GA_percent5_y80_epsilonbest-Z.nc')
ds_PLASIM_Z

In [ ]:
ds_PLASIM = xr.open_dataarray('../../PLASIM/committor/Skill-GA_percent5_y80_epsilonbest.nc')
ds_PLASIM

In [ ]:
ds_ERA = xr.open_dataarray('Skill-GA_percent5_epsilonbest.nc').sel(tau=ds_PLASIM['tau'])
ds_ERA

In [ ]:
plt.close(10)
fig, ax = plt.subplots(num=10, figsize=(9,6))

for i,T in enumerate([1,7,14]):
    
    ss = ds_ERA.sel(T=T)
    uplt.plot(ss['tau'].data + i*0.1, uplt.xr_avg(ss, 'fold').data.squeeze(), label=f'${T = }$', color=default_colors[i])
    
    ss = ds_PLASIM_Z.sel(T=T)
    uplt.plot(ss['tau'].data + i*0.1 + 0.05, uplt.xr_avg(ss, 'fold').data.squeeze(), color=default_colors[i], linestyle='dashed')
    
    ss = ds_PLASIM.sel(T=T)
    uplt.plot(ss['tau'].data + i*0.1 + 0.1, uplt.xr_avg(ss, 'fold').data.squeeze(), color=default_colors[i], linestyle='dotted')
    
plt.plot([0],[0], color='black', label='ERA5')
plt.plot([0],[0], color='black', linestyle='dashed', label='PlaSim-Z')
plt.plot([0],[0], color='black', linestyle='dotted', label='PlaSim-TZS')
    
plt.legend()
plt.xlabel(r'$\tau$ [days]')
plt.ylabel('Norm log score')

ax.set_xlim(-0.5,16)
ax.set_ylim(*ax.get_ylim())

# plt.axhline(0, color='black', linestyle='dashed')
# plt.axvline(0, color='black', alpha=0.2)
plt.fill_between(ax.get_xlim(), ax.get_ylim()[0], 0, color='red', alpha=0.2)

fig.tight_layout()

# fig.savefig(f'{HOME}/SvTtau_ERA-PlaSim.pdf')